In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
x_train = pd.read_csv('./data/track1/features/x_train_normal.csv')
x_valid = pd.read_csv('./data/track1/features/x_valid_normal.csv')
x_test = pd.read_csv('./data/track1/features/x_test_normal.csv')
y_train = pd.read_csv('./data/track1/features/y_train_normal.csv')
y_valid = pd.read_csv('./data/track1/features/y_valid_normal.csv')
y_test = pd.read_csv('./data/track1/features/y_test_normal.csv')